<a href="https://colab.research.google.com/github/KyraSebestyen/bioEmbedding/blob/main/fehlerhafteZeilenFinden.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas
import numpy
from tqdm.notebook import tqdm
import os
import gzip
import json
import seaborn
import matplotlib.pyplot
!pip install flair
from flair.data import Sentence
from flair.embeddings import TransformerWordEmbeddings
import torch

In [ ]:
# connect to google drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_trf-3.2.0/en_core_web_trf-3.2.0.tar.gz --no-deps

import spacy
!pip install spacy_transformers
import spacy_transformers
nlp = spacy.load('en_core_web_trf')
nlp.disable_pipes(["parser", "ner"])

     |████████████████████████████████| 460.2 MB 31 kB/s 


['parser', 'ner']

In [ ]:
def spacy_to_json(spacy_doc):
    doc_dict = spacy_doc.to_json()
    # print(doc_dict)
    sent_boundaries = [10000] # habe nur Sätze und brauche deshalb nicht Satzende
    doc_dict = doc_dict["tokens"]
    current_sentence = 0
    for i, t in enumerate(spacy_doc):
        doc_dict[i]["text"] = t.text
        if spacy_doc[i].is_sent_start:
            doc_dict[i]["is_sent_start"] = True
        else:
            doc_dict[i]["is_sent_start"] = False
        if doc_dict[i]["end"] > sent_boundaries[current_sentence]:
            current_sentence += 1
        doc_dict[i]["sentence_id"] = current_sentence
    return doc_dict

In [ ]:
def get_batches(lst, batch_size):
  for i in range(0, len(lst), batch_size): 
    yield lst[i : i + batch_size] # gibt stückweise Elemente der Liste (1 Batch pro Iteration)


from datetime import datetime

In [ ]:
corpus = pandas.read_csv('/content/gdrive/My Drive/Colab_Notebooks/Okkurrenzen Auszug.csv', sep = ';', quoting = 3) # 3 means QUOTE_NONE

In [ ]:
print(corpus.shape)
corpus = corpus[corpus.text.map(len) < 1000] # 17 Sätze sind länger
print(corpus.shape)

(20676, 6)
(20659, 6)
          OccId  ...                                               text
0        cattle  ...  the travellers visit a gentleman farmer, who i...
1         plant  ...  i then exhibited some of what i considered my ...
2          bird  ...  i next produced some rare insects, and feather...
3        insect  ...  i next produced some rare insects, and feather...
4         shark  ...  we had, in singular succession, dead calms and...
...         ...  ...                                                ...
20671      tree  ...  at length, for the first time during about hal...
20672     horse  ...  westport had beheld strange scenes, but a roug...
20673     horse  ...  here we were soon visited by a number of peopl...
20674  moccasin  ...  in the morning we hardly recognized each other...
20675     horse  ...            shaw had given him a horse at westport.

[20659 rows x 6 columns]


In [ ]:
valid = []
mwu = []
notFound = []

for index, line in corpus.iterrows():
  occId = line["OccId"]
  if len(occId.split(" ")) > 1 or len(occId.split("-")) > 1:
    mwu.append(line)
    continue
  doc = list(nlp.pipe([line["text"]]))[0]
  spacyList = spacy_to_json(doc)
  found = False
  for docJson in spacyList:
    if docJson["lemma"] == occId or occId in docJson["text"]:
      valid.append(line)
      found = True
      break
  if not found:
    notFound.append(line)

pandas.DataFrame(valid).to_csv('/content/gdrive/My Drive/Colab_Notebooks/Valid.csv', sep = ';', quoting = 3)
pandas.DataFrame(mwu).to_csv('/content/gdrive/My Drive/Colab_Notebooks/MWU.csv', sep = ';', quoting = 3)
pandas.DataFrame(notFound).to_csv('/content/gdrive/My Drive/Colab_Notebooks/NotFound.csv', sep = ';', quoting = 3)


/usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
